In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Gastroesophageal_reflux_disease_(GERD)"
cohort = "GSE77563"

# Input paths
in_trait_dir = "../../input/GEO/Gastroesophageal_reflux_disease_(GERD)"
in_cohort_dir = "../../input/GEO/Gastroesophageal_reflux_disease_(GERD)/GSE77563"

# Output paths
out_data_file = "../../output/preprocess/Gastroesophageal_reflux_disease_(GERD)/GSE77563.csv"
out_gene_data_file = "../../output/preprocess/Gastroesophageal_reflux_disease_(GERD)/gene_data/GSE77563.csv"
out_clinical_data_file = "../../output/preprocess/Gastroesophageal_reflux_disease_(GERD)/clinical_data/GSE77563.csv"
json_path = "../../output/preprocess/Gastroesophageal_reflux_disease_(GERD)/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# 1. Analyze gene expression data availability
# Based on the background information, this dataset contains Affymetrix expression array profiles
# which indicates it's likely to contain gene expression data
is_gene_available = True  # Affymetrix Human Gene 2.1 ST arrays data is available

# 2. Analyze variable availability and data type conversion
# 2.1 Data Availability

# For trait (GERD):
# Key 5 contains GERD status information
trait_row = 5

# For age:
# Key 1 contains age information
age_row = 1

# For gender:
# Key 2 contains gender information
gender_row = 2

# 2.2 Data Type Conversion

# For trait (GERD):
def convert_trait(value):
    if pd.isna(value):
        return None
    
    # Extract the value after the colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert to binary (0 for No GERD, 1 for GERD)
    if "No GERD" in value:
        return 0
    elif "GERD" in value:
        return 1
    else:
        return None

# For age:
def convert_age(value):
    if pd.isna(value):
        return None
    
    # Extract the value after the colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert to continuous
    try:
        return float(value)
    except:
        return None

# For gender:
def convert_gender(value):
    if pd.isna(value):
        return None
    
    # Extract the value after the colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert to binary (0 for female, 1 for male)
    if "female" in value.lower():
        return 0
    elif "male" in value.lower():
        return 1
    else:
        return None

# 3. Save Metadata
# Determine if trait data is available
is_trait_available = trait_row is not None

# Initial filtering and save metadata
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical data
    print("Preview of extracted clinical data:")
    print(preview_df(selected_clinical_df))
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Gene Data Extraction

In [ ]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


### Step 4: Gene Identifier Review

In [ ]:
# The identifiers seen in the gene expression data (16657436, 16657445, etc.) appear to be 
# Affymetrix probeset IDs rather than human gene symbols. These are numeric identifiers 
# specific to the microarray platform used in this study.
#
# Standard human gene symbols would typically be alphabetic (like "BRCA1", "TP53", etc.) 
# or alphanumeric combinations that follow a recognizable pattern.
#
# These numeric IDs will need to be mapped to standard gene symbols for meaningful analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [ ]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


### Step 6: Gene Identifier Mapping

In [ ]:
# 1. First, make sure we have the gene expression data loaded properly
gene_data = get_genetic_data(matrix_file)

# 2. Extract and prepare the gene mapping dataframe
# Looking at the data, 'ID' contains the numeric identifiers that match the gene expression data
# The 'gene_assignment' column contains gene symbol information
mapping_df = gene_annotation[['ID', 'gene_assignment']]

# Convert IDs to strings for consistent comparison
mapping_df['ID'] = mapping_df['ID'].astype(str)

# 3. Extract gene symbols from the complex gene_assignment field
# Use the built-in extract_human_gene_symbols function which is more robust
# It's able to identify gene symbols from complex text following standard patterns
mapping_df['Gene'] = mapping_df['gene_assignment'].apply(extract_human_gene_symbols)

# Print diagnostic information about the mapping
print(f"Number of probes in gene_data: {len(gene_data.index)}")
print(f"Original mapping rows: {len(mapping_df)}")
print(f"Sample of original mappings:")
print(mapping_df[['ID', 'Gene']].head(5))

# Filter out mappings with empty gene lists
mapping_df = mapping_df[mapping_df['Gene'].apply(len) > 0]
print(f"Filtered mapping rows: {len(mapping_df)}")
print(f"Number of mappable probes: {len(mapping_df[mapping_df['ID'].isin(gene_data.index)])}")

# 4. Apply the gene mapping to get gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df[['ID', 'Gene']])

# Check data before normalization
print(f"\nBefore normalization - shape: {gene_data.shape}")
print(f"Sample gene symbols before normalization: {list(gene_data.index[:10])}")

# 5. Create a modified normalization function that preserves unmapped symbols
def modified_normalize_gene_symbols(gene_df):
    with open("./metadata/gene_synonym.json", "r") as f:
        synonym_dict = json.load(f)
    
    # Create a mapping function that keeps original symbol if not in dictionary
    def map_symbol(x):
        return synonym_dict.get(x.upper(), x)
    
    gene_df.index = gene_df.index.map(map_symbol)
    # Group and average rows with same index (after normalization)
    return gene_df.groupby(gene_df.index).mean()

# Apply the modified normalization
gene_data = modified_normalize_gene_symbols(gene_data)

# Preview the mapped gene data
print("\nPreview of mapped gene data:")
print(f"Shape: {gene_data.shape}")
print(gene_data.head())

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


### Step 7: Data Normalization and Linking

In [ ]:
# 1. Examine the issue with gene mapping in previous step
print("Analyzing gene mapping issue...")
gene_data = get_genetic_data(matrix_file)
print(f"Gene data from file: {gene_data.shape}")

# Load gene annotation data more directly
gene_annotation = get_gene_annotation(soft_file)
print(f"Gene annotation data shape: {gene_annotation.shape}")

# Extract and prepare the gene mapping dataframe
mapping_df = gene_annotation[['ID', 'gene_assignment']]

# Check a sample of the gene_assignment column
print("\nSample of gene_assignment data:")
for i, assignment in enumerate(mapping_df['gene_assignment'].head(3)):
    print(f"Assignment {i}: {assignment[:100]}...")

# Apply extract_human_gene_symbols function to get gene symbols
mapping_df['Gene'] = mapping_df['gene_assignment'].apply(extract_human_gene_symbols)

# Print sample of extracted genes
print("\nSample of extracted genes:")
print(mapping_df[['ID', 'Gene']].head(5))

# Implement a direct gene mapping approach as a fallback
print("\nImplementing alternative gene mapping approach...")
gene_to_expr = {}

# First, create a mapping from probe IDs to gene symbols
probe_to_genes = {}
for i, row in mapping_df.iterrows():
    probe_id = str(row['ID'])
    genes = row['Gene']
    if genes and len(genes) > 0:
        probe_to_genes[probe_id] = genes

# Apply the mapping to distribute expression values
for probe_id, expr_values in gene_data.iterrows():
    probe_id_str = str(probe_id)
    if probe_id_str in probe_to_genes:
        genes = probe_to_genes[probe_id_str]
        value_share = 1.0 / len(genes)
        
        for gene in genes:
            if gene not in gene_to_expr:
                gene_to_expr[gene] = pd.Series(0, index=expr_values.index)
            gene_to_expr[gene] += expr_values * value_share

# Convert the dictionary to a DataFrame
gene_data = pd.DataFrame(gene_to_expr).T
print(f"Gene data after direct mapping: {gene_data.shape}")

# 1. Normalize gene symbols in the obtained gene expression data
# Normalize gene symbols using NCBI Gene database
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
print(f"Sample gene symbols after normalization: {list(normalized_gene_data.index[:10])}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Link clinical and genetic data
# Get raw clinical data from the matrix file
_, clinical_raw = get_background_and_clinical_data(matrix_file)

# Extract clinical features using the defined conversion functions
clinical_features = geo_select_clinical_features(
    clinical_df=clinical_raw,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

print("Clinical features shape:", clinical_features.shape)

# Save clinical features to file
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features.to_csv(out_clinical_data_file)
print(f"Clinical features saved to {out_clinical_data_file}")

# Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, first 5 columns):")
if linked_data.shape[1] >= 5:
    print(linked_data.iloc[:5, :5])
else:
    print(linked_data.head())

# 3. Handle missing values
print("\nMissing values before handling:")
print(f"  Trait ({trait}) missing: {linked_data[trait].isna().sum()} out of {len(linked_data)}")
if 'Age' in linked_data.columns:
    print(f"  Age missing: {linked_data['Age'].isna().sum()} out of {len(linked_data)}")
if 'Gender' in linked_data.columns:
    print(f"  Gender missing: {linked_data['Gender'].isna().sum()} out of {len(linked_data)}")

gene_cols = [col for col in linked_data.columns if col not in [trait, 'Age', 'Gender']]
if gene_cols:
    print(f"  Genes with >20% missing: {sum(linked_data[gene_cols].isna().mean() > 0.2)}")
    print(f"  Samples with >5% missing genes: {sum(linked_data[gene_cols].isna().mean(axis=1) > 0.05)}")

cleaned_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {cleaned_data.shape}")

# 4. Evaluate bias in trait and demographic features
is_trait_biased = False
if len(cleaned_data) > 0:
    trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait)
    is_trait_biased = trait_biased
else:
    print("No data remains after handling missing values.")
    is_trait_biased = True

# 5. Final validation and save
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=len(normalized_gene_data) > 0, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=cleaned_data,
    note=f"Dataset contains gene expression data for {trait} analysis."
)

# 6. Save if usable
if is_usable and len(cleaned_data) > 0:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    cleaned_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable or empty and was not saved")